### Выбор Retrieval подхода для последующего построения саммари

In [1]:
! pip install scikit-learn rank_bm25

Looking in indexes: https://artifactory.tcsbank.ru/artifactory/api/pypi/python-all/simple

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [1]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [2]:
import os, torch, gc
print("PID:", os.getpid())
print("allocated GB:", torch.cuda.memory_allocated()/1024**3)
print("reserved  GB:", torch.cuda.memory_reserved()/1024**3)


PID: 1443
allocated GB: 0.0
reserved  GB: 0.0


In [3]:
import pandas as pd

df = pd.read_csv("../data/cleaned_news_exp.csv")[["message_id", "id_channel", "message", "date", "topic", "viral_final"]]
df.head()

,message_id,id_channel,message,date,topic,viral_final
0,0007e2f8-787d-404f-91ff-e2582096a4a7,18,Сербия согласна поддержать санкции Евросоюза п...,2025-07-26 07:01:09,Санкции и геополитика,0.703622
1,000884a5-8291-4ec1-805f-ac131112aaf7,6,Китайский рынок акций упал сильнее всего с апр...,2025-09-04 10:16:56,Рынки капитала,0.680369
2,000b0331-92a9-4eb4-9f58-d00811257758,18,Министерство труда США отменило рекомендации 2...,2025-05-29 04:05:09,Государственная экономическая политика,0.589152
3,000b8df7-d902-41eb-b668-900614902f0a,6,Чистая прибыль Московской биржи по МСФО во вто...,2025-08-26 11:40:55,Корпоративные финансы,0.461692
4,0011adea-7a98-4dcc-b753-905597b42788,4,"США хотят получить нефть и «всё, что угодно» о...",2025-02-22 20:57:18,Сырьевые рынки,0.504362


In [4]:
df[df["message_id"] == "588a6960-aa5b-406b-8417-fdad3ccde1e9"].iloc[0]["message"]

'Кофе в России может подорожать на 50% уже в этом году, предупреждают аналитики Last Wish. Причины — засуха в Бразилии, рост логистических расходов, дефицит запасов и разбалансировка мирового спроса. Больше всего прибавит в цене арабика.\n\n🔵 Bloomberg'

In [5]:
import re
import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import List, Dict, Optional, Tuple

In [6]:
def ensure_datetime(df: pd.DataFrame, col: str = "date") -> pd.DataFrame:
    d = df.copy()
    d[col] = pd.to_datetime(d[col], utc=True, errors="coerce")
    d = d.dropna(subset=[col])
    d["date_day"] = d[col].dt.floor("D")
    return d

def normalize_text(s: str) -> str:
    s = s or ""
    s = re.sub(r"\s+", " ", s).strip()
    return s

def l2_normalize(x: np.ndarray, eps: float = 1e-12) -> np.ndarray:
    n = np.linalg.norm(x, axis=1, keepdims=True)
    return x / np.maximum(n, eps)

def cosine_scores(query_vec: np.ndarray, doc_mat_l2: np.ndarray) -> np.ndarray:
    q = l2_normalize(query_vec.reshape(1, -1))[0]
    return doc_mat_l2 @ q

def topk_from_scores(scores: np.ndarray, k: int) -> Tuple[np.ndarray, np.ndarray]:
    k = min(k, len(scores))
    if k == len(scores):
        idx = np.argsort(-scores)
    else:
        idx = np.argpartition(-scores, kth=k-1)[:k]
        idx = idx[np.argsort(-scores[idx])]
    return idx, scores[idx]

In [7]:
import numpy as np
from sentence_transformers import SentenceTransformer
from pathlib import Path

_E5_MODEL_NAME = "intfloat/multilingual-e5-large"
_e5_model = None

def _get_e5():
    global _e5_model
    if _e5_model is None:
        _e5_model = SentenceTransformer(_E5_MODEL_NAME)
    return _e5_model

class Encoder:
    def __init__(self, default_prefix: str = "query: "):
        self.default_prefix = default_prefix

    def _ensure_prefix(self, t: str) -> str:
        t = t or ""
        if t.startswith("query:") or t.startswith("passage:"):
            return t
        return (self.default_prefix + t) if self.default_prefix else t

    def encode_texts(self, texts):
        mod = _get_e5()
        texts = [self._ensure_prefix(x) for x in texts]
        vecs = mod.encode(texts, normalize_embeddings=True, show_progress_bar=False)
        return vecs.astype(np.float32)

def encode_query(query_text: str) -> np.ndarray:
    enc = Encoder(default_prefix="query: ")
    return enc.encode_texts([query_text])[0]


def load_and_normalize_emb(path: Path) -> np.ndarray:
    E = np.load(path, mmap_mode="r")
    X = E.astype(np.float32)
    X /= (np.linalg.norm(X, axis=1, keepdims=True) + 1e-12)
    return X

E_docs = load_and_normalize_emb(Path("embeddings/emb_e5_large_fp16.npy"))
encoder = Encoder(default_prefix="query: ")


## Semantic retrieve

In [8]:
from typing import Tuple, Optional
import pandas as pd
import numpy as np

def semantic_retrieve(
    df: pd.DataFrame,
    E_docs_l2: np.ndarray,
    query: str,
    encoder: Encoder,
    k: int = 200,
    end_date: Optional[pd.Timestamp] = None,
) -> pd.DataFrame:
    
    d = ensure_datetime(df, "date")
    if end_date is None:
        end_date = d["date_day"].max()
    end_date = pd.to_datetime(end_date, utc=True)
    idx_pool = np.arange(len(d))

    qv = encoder.encode_texts([f"query: {query}"])[0]
    scores = cosine_scores(qv, E_docs_l2[idx_pool])
    local_idx, local_scores = topk_from_scores(scores, k=min(k, len(idx_pool)))
    glob_idx = idx_pool[local_idx]

    out = d.iloc[glob_idx].copy()
    out["sim"] = local_scores
    return out.sort_values("sim", ascending=False).reset_index(drop=True)

In [9]:
cand = semantic_retrieve(
    df=df,
    E_docs_l2=E_docs,
    query="Кофе в России может подорожать на 50% уже в этом году, предупреждают аналитики Last Wish. Причины — засуха в Бразилии, рост логистических расходов, дефицит запасов и разбалансировка мирового спроса. Больше всего прибавит в цене арабика.\n\n🔵 Bloomberg",
    encoder=encoder,
    k=200,
)

cand[["date_day","id_channel","message_id","sim","message", "topic"]].head(20)


,date_day,id_channel,message_id,sim,message,topic
0,2025-04-07 00:00:00+00:00,5,588a6960-aa5b-406b-8417-fdad3ccde1e9,0.945688,Кофе в России может подорожать на 50% уже в эт...,Сырьевые рынки
1,2025-03-24 00:00:00+00:00,4,4f13a7ff-e768-43ef-a71b-e429f3dada1b,0.880965,Кофе может резко подорожать до 40% уже в этом ...,Сырьевые рынки
2,2025-08-01 00:00:00+00:00,5,8fbc1ae5-130c-4773-ba34-5e004af5861e,0.879588,В России ожидается новое подорожание кофе — на...,Сырьевые рынки
3,2025-07-10 00:00:00+00:00,4,5498bb88-6944-4a0b-9a19-324fc0b16c2e,0.879148,Кофе подорожает на 40% в этом году — прогноз э...,Сырьевые рынки
4,2025-03-24 00:00:00+00:00,5,6ba48c2b-b64b-4326-aa39-05ebd207ca59,0.871201,Россиян предупредили о резком скачке цен на ко...,Сырьевые рынки
5,2025-01-30 00:00:00+00:00,5,746d754f-a71f-4717-a59c-a0601ac0fe16,0.863449,Цены на кофе в России подскочат ещё на 20% уже...,Сырьевые рынки
6,2025-01-30 00:00:00+00:00,6,947db7a2-0784-4980-8ef5-8b2f7d2bca9c,0.859806,Крупнейшие поставщики собираются увеличить отп...,Сырьевые рынки
7,2025-03-04 00:00:00+00:00,4,52e1439a-fe85-4305-a407-0bfc2b873efa,0.857452,Кофе рекордно подорожает на 30-40% уже к концу...,Сырьевые рынки
8,2025-03-17 00:00:00+00:00,4,a6a9c0a2-93e7-40ee-b84f-92ae43a515be,0.852232,Чай резко подорожает в России до 40% уже в 202...,Сырьевые рынки
9,2025-01-09 00:00:00+00:00,18,f7212d03-4fc9-4c70-b58c-bfb3336ff4ca,0.850911,Цены на кофе и шоколад в этом году взлетят на ...,Сырьевые рынки


## BM25 retrieve

In [10]:
from typing import Optional
import numpy as np
import pandas as pd
import re
from rank_bm25 import BM25Okapi

def _tok(s: str):
    return re.findall(r"[A-Za-zА-Яа-яЁё0-9_]+", (s or "").lower())

def build_bm25_index(df: pd.DataFrame, text_col: str = "message"):
    texts = df[text_col].fillna("").astype(str).tolist()
    tokenized = [_tok(t) for t in texts]
    bm25 = BM25Okapi(tokenized)
    return bm25

def bm25_retrieve(
    df: pd.DataFrame,
    bm25,
    query: str,
    k: int = 200,
    end_date: Optional[pd.Timestamp] = None,
) -> pd.DataFrame:
    d = ensure_datetime(df, "date")
    if "channel" not in d.columns and "id_channel" in d.columns:
        d = d.rename(columns={"id_channel": "channel"})
    if end_date is None:
        end_date = d["date_day"].max()
    end_date = pd.to_datetime(end_date, utc=True)

    idx_pool = np.arange(len(d))

    q = _tok(query)
    scores_full = np.asarray(bm25.get_scores(q), dtype=np.float32)
    scores = scores_full[idx_pool]
    local_idx, local_scores = topk_from_scores(scores, k=min(k, len(idx_pool)))
    glob_idx = idx_pool[local_idx]

    out = d.iloc[glob_idx].copy()
    out["bm25"] = local_scores
    return out.sort_values("bm25", ascending=False).reset_index(drop=True)

In [11]:
bm25 = build_bm25_index(df, text_col="message")
cand_bm25 = bm25_retrieve(df, bm25, "Кофе в России может подорожать на 50% уже в этом году, предупреждают аналитики Last Wish. Причины — засуха в Бразилии, рост логистических расходов, дефицит запасов и разбалансировка мирового спроса. Больше всего прибавит в цене арабика.\n\n🔵 Bloomberg", k=200)
cand_bm25[["date_day","channel","message_id","bm25","message", "topic"]].head(20)

,date_day,channel,message_id,bm25,message,topic
0,2025-04-07 00:00:00+00:00,5,588a6960-aa5b-406b-8417-fdad3ccde1e9,200.892044,Кофе в России может подорожать на 50% уже в эт...,Сырьевые рынки
1,2025-03-24 00:00:00+00:00,4,4f13a7ff-e768-43ef-a71b-e429f3dada1b,82.025490,Кофе может резко подорожать до 40% уже в этом ...,Сырьевые рынки
2,2025-03-24 00:00:00+00:00,5,6ba48c2b-b64b-4326-aa39-05ebd207ca59,62.448727,Россиян предупредили о резком скачке цен на ко...,Сырьевые рынки
3,2025-05-31 00:00:00+00:00,5,e34759e9-a8c9-48e8-8ac0-e9fafdfa74a4,49.746479,Бензин в России начнёт дорожать в два раза быс...,Сырьевые рынки
4,2025-01-09 00:00:00+00:00,18,f7212d03-4fc9-4c70-b58c-bfb3336ff4ca,49.626266,Цены на кофе и шоколад в этом году взлетят на ...,Сырьевые рынки
5,2025-06-16 00:00:00+00:00,3,d009f7ce-5386-4530-a117-f8c70c23288f,48.738232,Кофе и какао в мире могут подорожать из-за тор...,Сырьевые рынки
6,2025-08-14 00:00:00+00:00,18,1c2253b9-4b34-44bd-9ddf-1d290c6fa513,47.065208,Международное энергетическое агентство снизило...,Сырьевые рынки
7,2025-03-17 00:00:00+00:00,5,0ee0c3ae-98e7-4dbb-b1f9-8ce34c26b616,46.789661,"Чай подорожает в 2025 году. Элитные сорта, вкл...",Другое
8,2025-08-30 00:00:00+00:00,1,a1899d8e-2f4c-4b11-848c-c95f8465c106,46.733784,По итогам августа пиломатериалы и медь больше ...,Сырьевые рынки
9,2025-07-11 00:00:00+00:00,6,9f0c4216-c0f7-4d9d-99e5-d89705495053,45.063896,Повышение до 50% импортных пошлин США на товар...,Государственная экономическая политика


## Hybrid

In [12]:
from typing import Optional
import numpy as np
import pandas as pd

def rrf_fuse_weighted(sem_idx, bm_idx, k_rrf=60, w_sem=1.0, w_bm=1.0):
    scores = {}
    for w, rank in ((w_sem, sem_idx), (w_bm, bm_idx)):
        for i, doc_id in enumerate(rank):
            scores[doc_id] = scores.get(doc_id, 0.0) + w * (1.0 / (k_rrf + i + 1))
    ids = np.array(sorted(scores.keys(), key=lambda x: -scores[x]), dtype=int)
    vals = np.array([scores[i] for i in ids], dtype=np.float32)
    return ids, vals


def hybrid_retrieve(
    df: pd.DataFrame,
    E_docs_l2: np.ndarray,
    bm25,
    query: str,
    encoder: Encoder,
    k: int = 200,
    topN_each: int = 1000,
    k_rrf: int = 60,
    w_sem: float = 1.0,
    w_bm: float = 1.0,
    end_date: Optional[pd.Timestamp] = None,
) -> pd.DataFrame:
    d = ensure_datetime(df, "date")
    if "channel" not in d.columns and "id_channel" in d.columns:
        d = d.rename(columns={"id_channel": "channel"})
    if end_date is None:
        end_date = d["date_day"].max()
    end_date = pd.to_datetime(end_date, utc=True)

    idx_pool = np.arange(len(d))

    qv = encoder.encode_texts([f"query: {query}"])[0]
    sem_scores = cosine_scores(qv, E_docs_l2[idx_pool])
    sem_local, _ = topk_from_scores(sem_scores, k=min(topN_each, len(idx_pool)))
    sem_rank = idx_pool[sem_local].tolist()

    q = _tok(query)
    bm_scores_full = np.asarray(bm25.get_scores(q), dtype=np.float32)
    bm_scores = bm_scores_full[idx_pool]
    bm_local, _ = topk_from_scores(bm_scores, k=min(topN_each, len(idx_pool)))
    bm_rank = idx_pool[bm_local].tolist()

    fused_ids, fused_scores = rrf_fuse_weighted(
        sem_rank, bm_rank, k_rrf=k_rrf, w_sem=w_sem, w_bm=w_bm
    )
    fused_ids = fused_ids[:min(k, len(fused_ids))]
    fused_scores = fused_scores[:len(fused_ids)]

    out = d.iloc[fused_ids].copy()
    out["rrf"] = fused_scores
    out["sem_sim"] = cosine_scores(qv, E_docs_l2[fused_ids])
    out["bm25"] = bm_scores_full[fused_ids]
    return out.sort_values("rrf", ascending=False).reset_index(drop=True)



In [13]:
cand_h = hybrid_retrieve(
    df=df,
    E_docs_l2=E_docs,
    bm25=bm25,
    query="Кофе в России может подорожать на 50% уже в этом году, предупреждают аналитики Last Wish. Причины — засуха в Бразилии, рост логистических расходов, дефицит запасов и разбалансировка мирового спроса. Больше всего прибавит в цене арабика.\n\n🔵 Bloomberg",
    encoder=encoder,
    k=200,
    topN_each=2000,
    k_rrf=10,
    w_sem=1.2,
    w_bm=0.5,
)


cand_h[["date_day","channel","message_id","rrf","sem_sim","bm25","message", "topic"]].head(20)

,date_day,channel,message_id,rrf,sem_sim,bm25,message,topic
0,2025-04-07 00:00:00+00:00,5,588a6960-aa5b-406b-8417-fdad3ccde1e9,0.154545,0.945688,200.892044,Кофе в России может подорожать на 50% уже в эт...,Сырьевые рынки
1,2025-03-24 00:00:00+00:00,4,4f13a7ff-e768-43ef-a71b-e429f3dada1b,0.141667,0.880965,82.025490,Кофе может резко подорожать до 40% уже в этом ...,Сырьевые рынки
2,2025-03-24 00:00:00+00:00,5,6ba48c2b-b64b-4326-aa39-05ebd207ca59,0.118462,0.871201,62.448727,Россиян предупредили о резком скачке цен на ко...,Сырьевые рынки
3,2025-08-01 00:00:00+00:00,5,8fbc1ae5-130c-4773-ba34-5e004af5861e,0.113141,0.879588,43.442513,В России ожидается новое подорожание кофе — на...,Сырьевые рынки
4,2025-07-10 00:00:00+00:00,4,5498bb88-6944-4a0b-9a19-324fc0b16c2e,0.100866,0.879148,40.782764,Кофе подорожает на 40% в этом году — прогноз э...,Сырьевые рынки
5,2025-01-30 00:00:00+00:00,5,746d754f-a71f-4717-a59c-a0601ac0fe16,0.096739,0.863449,43.555683,Цены на кофе в России подскочат ещё на 20% уже...,Сырьевые рынки
6,2025-01-09 00:00:00+00:00,18,f7212d03-4fc9-4c70-b58c-bfb3336ff4ca,0.093333,0.850911,49.626266,Цены на кофе и шоколад в этом году взлетят на ...,Сырьевые рынки
7,2025-06-16 00:00:00+00:00,3,d009f7ce-5386-4530-a117-f8c70c23288f,0.081250,0.843620,48.738232,Кофе и какао в мире могут подорожать из-за тор...,Сырьевые рынки
8,2025-01-30 00:00:00+00:00,6,947db7a2-0784-4980-8ef5-8b2f7d2bca9c,0.076686,0.859806,35.191982,Крупнейшие поставщики собираются увеличить отп...,Сырьевые рынки
9,2025-04-19 00:00:00+00:00,3,ce0ff05d-0916-4fd0-b197-b7b9e5fcc923,0.075983,0.844805,44.961369,Колумбийский эксперт и бариста Себастьян Сулуа...,Сырьевые рынки


## Посравниваем

In [14]:
import json, re, torch
from transformers import AutoTokenizer, AutoModelForCausalLM

JUDGE_MODEL = "Qwen/Qwen2.5-7B-Instruct"

judge_tokenizer = AutoTokenizer.from_pretrained(JUDGE_MODEL, trust_remote_code=True)
judge_model = AutoModelForCausalLM.from_pretrained(
    JUDGE_MODEL,
    torch_dtype=torch.bfloat16,
    device_map="cuda",
    trust_remote_code=True,
)
judge_model.eval()

def _extract_first_json_obj(text: str):
    text = (text or "").strip()
    if text.startswith("{") and text.endswith("}"):
        try:
            return json.loads(text)
        except Exception:
            pass
    for m in re.finditer(r"\{.*?\}", text, flags=re.DOTALL):
        cand = m.group(0)
        try:
            return json.loads(cand)
        except Exception:
            continue
    return None


@torch.inference_mode()
def judge_pairs_batched(pairs, batch_size=32, max_new_tokens=40):
    out_scores = []
    for i in range(0, len(pairs), batch_size):
        chunk = pairs[i:i+batch_size]

        prompts = []
        for q, sn in chunk:
            user = f"Запрос:\n{q}\n\nДокумент:\n{sn}\n"
            messages = [
                {"role": "system", "content": JUDGE_SYSTEM},
                {"role": "user", "content": user},
            ]
            prompts.append(
                judge_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            )

        enc = judge_tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
        ).to(judge_model.device)

        gen_ids = judge_model.generate(
            **enc,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            eos_token_id=judge_tokenizer.eos_token_id,
            pad_token_id=judge_tokenizer.eos_token_id,
        )

        for b in range(len(chunk)):
            prompt_len = int(enc["attention_mask"][b].sum().item())
            gen_txt = judge_tokenizer.decode(gen_ids[b][prompt_len:], skip_special_tokens=True)
            data = _extract_first_json_obj(gen_txt)
            rel = int(data["relevance"]) if isinstance(data, dict) and data.get("relevance") in (0,1,2) else 0
            out_scores.append(rel)

    return out_scores

`torch_dtype` is deprecated! Use `dtype` instead!


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [15]:
JUDGE_SYSTEM = """Ты — строгий эксперт по информационному поиску по новостям (в т.ч. экономическим).

Твоя задача: оценить релевантность кандидатной новости запросу. Запрос может быть:
- коротким топиком (например "курс рубля к доллару"),
- или текстом другой новости (тогда запрос описывает конкретный инфоповод).

Используй ТОЛЬКО текст кандидатного документа. Ничего не додумывай.

Шкала релевантности:
2 — документ явно про то же самое: отвечает топику ИЛИ описывает тот же инфоповод/факт/событие, что и запрос.
1 — документ связан по теме/контексту, но это немного другой инфоповод, но про одно и то же или нет прямого соответствия запросу.
0 — нерелевантно совсем.

Правило строгости:
ставь 2 только если связь очевидна по тексту документа; если информации недостаточно — ставь 0 или 1.

Верни строго валидный JSON и ничего больше:
{"relevance": 0|1|2}
"""


In [16]:
import re
import numpy as np
import pandas as pd
from tqdm import tqdm

K = 50
SEED = 42
N_MANUAL = 20
N_AUTO = 20

manual_queries = list(dict.fromkeys([
    "рост цен на кофе",
    "ключевая ставка цб решение",
    "повышение ключевой ставки",
    "снижение ключевой ставки",
    "инфляция ускорилась",
    "инфляция замедлилась",
    "индекс потребительских цен ипц",
    "курс рубля к доллару",
    "курс евро к рублю",
    "ослабление рубля причины",
    "нефть brent падение",
    "нефть brent рост",
    "цена нефти brent прогноз",
    "газ в европе цены",
    "газ в европе прогноз",
    "мосбиржа индекс imoex рост",
    "мосбиржа индекс imoex падение",
    "санкции влияние на экономику",
    "санкции против россии новые",
    "ограничения на экспорт влияние",
]))

def clean_auto_query(t: str) -> str:
    t = re.sub(r"#\w+", " ", t)
    t = re.sub(r"[⚡️📈📉🇷🇺✅❗️🔥⬛ ⬜ ⚫ ⚪🔹]+", " ", t)
    t = re.sub(r"\s+", " ", t).strip()
    return t[:180]

def snippet(t: str, n: int = 800) -> str:
    t = re.sub(r"\s+", " ", (t or "").strip())
    return t[:n]

def dcg(rels):
    rels = np.array(rels, dtype=float)
    denom = np.log2(np.arange(2, len(rels) + 2))
    return float(np.sum((2**rels - 1) / denom))

def ndcg_at_k(rels, k=50):
    rels = list(rels)[:k]
    ideal = sorted(rels, reverse=True)
    denom = dcg(ideal)
    return 0.0 if denom == 0 else dcg(rels) / denom

def precision_at_k(rels, k=50, thr=1):
    rels = np.array(list(rels)[:k])
    return float(np.mean(rels >= thr)) if len(rels) else 0.0

def mrr_at_k(rels, k=50, thr=2):
    rels = list(rels)[:k]
    for i, r in enumerate(rels, start=1):
        if r >= thr:
            return 1.0 / i
    return 0.0


FULL_QUERY_CHARS = 1500

def clean_auto_query_full(t: str, n: int = FULL_QUERY_CHARS) -> str:
    t = re.sub(r"#\w+", " ", t)
    t = re.sub(r"[⚡️📈📉🇷🇺✅❗️🔥⬛⬜⚫⚪🔹]+", " ", t)
    t = re.sub(r"\s+", " ", t).strip()
    return t[:n]

rng = np.random.default_rng(SEED)

pool_df = df.copy()
pool_df["message"] = pool_df["message"].fillna("").astype(str)
pool_df = pool_df[pool_df["message"].str.len() >= 80]

sample = pool_df.sample(n=min(N_AUTO, len(pool_df)), random_state=SEED)

auto_queries = []
for _, row in sample.iterrows():
    auto_queries.append({
        "type": "news",
        "text": clean_auto_query_full(row["message"], n=FULL_QUERY_CHARS),
        "exclude_message_id": str(row["message_id"]),
    })

manual_q = [
    {"type": "topic", "text": q, "exclude_message_id": None}
    for q in manual_queries[:N_MANUAL]
]

queries = manual_q + auto_queries
queries = queries[: (N_MANUAL + N_AUTO)]

len(queries), queries


(40,
 [{'type': 'topic', 'text': 'рост цен на кофе', 'exclude_message_id': None},
  {'type': 'topic',
   'text': 'ключевая ставка цб решение',
   'exclude_message_id': None},
  {'type': 'topic',
   'text': 'повышение ключевой ставки',
   'exclude_message_id': None},
  {'type': 'topic',
   'text': 'снижение ключевой ставки',
   'exclude_message_id': None},
  {'type': 'topic', 'text': 'инфляция ускорилась', 'exclude_message_id': None},
  {'type': 'topic',
   'text': 'инфляция замедлилась',
   'exclude_message_id': None},
  {'type': 'topic',
   'text': 'индекс потребительских цен ипц',
   'exclude_message_id': None},
  {'type': 'topic',
   'text': 'курс рубля к доллару',
   'exclude_message_id': None},
  {'type': 'topic', 'text': 'курс евро к рублю', 'exclude_message_id': None},
  {'type': 'topic',
   'text': 'ослабление рубля причины',
   'exclude_message_id': None},
  {'type': 'topic', 'text': 'нефть brent падение', 'exclude_message_id': None},
  {'type': 'topic', 'text': 'нефть brent р

In [17]:
def _take_top_docs(cand: pd.DataFrame, k: int):
    cand = cand.copy()
    if "channel" not in cand.columns and "id_channel" in cand.columns:
        cand = cand.rename(columns={"id_channel": "channel"})
    cand = cand.head(k)
    ids = cand["message_id"].astype(str).tolist()
    docs = [snippet(x) for x in cand["message"].fillna("").astype(str).tolist()]
    return ids, docs

def eval_three_retrievers(
    df: pd.DataFrame,
    E_docs: np.ndarray,
    encoder,
    bm25,
    queries,
    k: int = 50,
    topN_each: int = 1000,
    k_rrf: int = 60,
    w_sem: float = 1.0,
    w_bm: float = 1.0,
    batch_size: int = 32,
):
    metric_rows = []
    pair_rows = []

    for qobj in tqdm(queries, desc="Queries"):
        q = qobj["text"]
        q_type = qobj.get("type", "unknown")
        exclude_mid = qobj.get("exclude_message_id", None)

        q_for_judge = q

        cand_sem = semantic_retrieve(df, E_docs, q, encoder, k=k)
        cand_bm  = bm25_retrieve(df, bm25, q, k=k)
        cand_hyb = hybrid_retrieve(
            df, E_docs, bm25, q, encoder,
            k=k,
            topN_each=topN_each,
            k_rrf=k_rrf,
            w_sem=w_sem,
            w_bm=w_bm
        )

        packs = [("semantic", cand_sem), ("bm25", cand_bm), ("hybrid", cand_hyb)]

        for method, cand in packs:
            if exclude_mid is not None and "message_id" in cand.columns:
                cand = cand[cand["message_id"].astype(str) != str(exclude_mid)]

            ids, docs = _take_top_docs(cand, k)
            rels = judge_pairs_batched([(q_for_judge, d) for d in docs], batch_size=batch_size)

            metric_rows.append({
                "q_type": q_type,
                "query": q_for_judge,
                "method": method,
                "ndcg@50": ndcg_at_k(rels, k),
                "mrr@50(rel=2)": mrr_at_k(rels, k, thr=2),
                "p@50(rel=2)": precision_at_k(rels, k, thr=2),
                "p@50(rel>=1)": precision_at_k(rels, k, thr=1),
                "n_docs": len(rels),
            })

            for r, (mid, rel) in enumerate(zip(ids, rels), start=1):
                pair_rows.append({
                    "q_type": q_type,
                    "query": q_for_judge,
                    "method": method,
                    "rank": r,
                    "message_id": mid,
                    "relevance": int(rel),
                })

    return pd.DataFrame(metric_rows), pd.DataFrame(pair_rows)



metrics_df, pairs_df = eval_three_retrievers(
    df=df,
    E_docs=E_docs,
    encoder=encoder,
    bm25=bm25,
    queries=queries,
    k=50,
    topN_each=2000,
    k_rrf=60,
    w_sem=1.0,
    w_bm=1.0,
    batch_size=32,
)


Queries: 100%|██████████| 40/40 [08:14<00:00, 12.37s/it]


In [18]:
print("allocated:", torch.cuda.memory_allocated()/1024**3, "GB")
print("reserved :", torch.cuda.memory_reserved()/1024**3, "GB")


allocated: 16.279865741729736 GB
reserved : 24.306640625 GB


In [19]:
summary = metrics_df.groupby("method", as_index=False)[["ndcg@50","mrr@50(rel=2)","p@50(rel=2)","p@50(rel>=1)"]].mean()
summary

,method,ndcg@50,mrr@50(rel=2),p@50(rel=2),p@50(rel>=1)
0,bm25,0.739282,0.521766,0.195867,0.385082
1,hybrid,0.783114,0.593155,0.267857,0.469888
2,semantic,0.763042,0.572750,0.280347,0.484347


In [20]:
pivot_ndcg = metrics_df.pivot(index="query", columns="method", values="ndcg@50")
pivot_mrr  = metrics_df.pivot(index="query", columns="method", values="mrr@50(rel=2)")
pivot_p2   = metrics_df.pivot(index="query", columns="method", values="p@50(rel=2)")

win_ndcg = {
    "hybrid>semantic": float((pivot_ndcg["hybrid"] > pivot_ndcg["semantic"]).mean()),
    "hybrid>bm25": float((pivot_ndcg["hybrid"] > pivot_ndcg["bm25"]).mean()),
    "semantic>bm25": float((pivot_ndcg["semantic"] > pivot_ndcg["bm25"]).mean()),
}
win_mrr = {
    "hybrid>semantic": float((pivot_mrr["hybrid"] > pivot_mrr["semantic"]).mean()),
    "hybrid>bm25": float((pivot_mrr["hybrid"] > pivot_mrr["bm25"]).mean()),
    "semantic>bm25": float((pivot_mrr["semantic"] > pivot_mrr["bm25"]).mean()),
}
win_p2 = {
    "hybrid>semantic": float((pivot_p2["hybrid"] > pivot_p2["semantic"]).mean()),
    "hybrid>bm25": float((pivot_p2["hybrid"] > pivot_p2["bm25"]).mean()),
    "semantic>bm25": float((pivot_p2["semantic"] > pivot_p2["bm25"]).mean()),
}

win_ndcg, win_mrr, win_p2

({'hybrid>semantic': 0.525, 'hybrid>bm25': 0.75, 'semantic>bm25': 0.625},
 {'hybrid>semantic': 0.15, 'hybrid>bm25': 0.225, 'semantic>bm25': 0.225},
 {'hybrid>semantic': 0.175, 'hybrid>bm25': 0.525, 'semantic>bm25': 0.525})

In [21]:
by_type = (metrics_df
           .groupby(["q_type", "method"])[["ndcg@50","mrr@50(rel=2)","p@50(rel=2)","p@50(rel>=1)"]]
           .mean()
           .sort_index())
display(by_type)

ndcg@50  mrr@50(rel=2)  p@50(rel=2)  p@50(rel>=1)
q_type method                                                      
news   bm25      0.650798       0.329167     0.036735      0.158163
       hybrid    0.660490       0.287500     0.035714      0.188776
       semantic  0.612732       0.239251     0.034694      0.184694
topic  bm25      0.827766       0.714365     0.355000      0.612000
       hybrid    0.905737       0.898810     0.500000      0.751000
       semantic  0.913351       0.906250     0.526000      0.784000

### Выводы:
- Необходима дополнительная фильтрация на релевантность кандидатов
- hybrid показывает сопоставимые результаты с dense retrieval для топиков, но bm25 спасает при кандидатогенерации для текстов

Принято решение использовать гибрид

## Проверяем насколько хуже с темпоральной составляющей

In [4]:
! pip install openpyxl pandas numpy rank_bm25 faiss-cpu transformers torch sentence-transformers accelerate vllm

Looking in indexes: https://artifactory.tcsbank.ru/artifactory/api/pypi/python-all/simple
  Using cached https://artifactory.tcsbank.ru/artifactory/api/pypi/python-all/packages/packages/openpyxl/3.1.5/openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
  Using cached https://artifactory.tcsbank.ru/artifactory/api/pypi/python-all/packages/packages/rank-bm25/0.2.2/rank_bm25-0.2.2-py3-none-any.whl (8.6 kB)
  Using cached https://artifactory.tcsbank.ru/artifactory/api/pypi/python-all/packages/packages/faiss-cpu/1.13.2/faiss_cpu-1.13.2-cp310-abi3-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (23.8 MB)
  Using cached https://artifactory.tcsbank.ru/artifactory/api/pypi/python-all/packages/packages/transformers/4.57.3/transformers-4.57.3-py3-none-any.whl (12.0 MB)
  Using cached https://artifactory.tcsbank.ru/artifactory/api/pypi/python-all/packages/packages/torch/2.9.1/torch-2.9.1-cp310-cp310-manylinux_2_28_x86_64.whl (899.8 MB)
  Using cached https://artifactory.tcsbank.ru/artifactory/api/pypi/py

In [24]:
import pandas as pd
import numpy as np

channels = pd.read_excel("../data/tg_channels.xlsx")
# df = pd.read_csv("../data/cleaned_news_exp.csv")[["message_id", "id_channel", "message", "date", "topic"]]
#df = pd.read_parquet("../data/tg_news_full.parquet")[["message_id", "id_channel", "message", "date"]]

/home/mlcore/conda/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [25]:
def attach_channel_w(
    df: pd.DataFrame,
    channels_df: pd.DataFrame,
    news_id_col: str = "id_channel",
    chan_id_col: str = "id",
    subs_col: str = "subscribers",
    weight_col: str = "channel_w",) -> pd.DataFrame:
    ch = channels_df[[chan_id_col, subs_col]].copy()
    ch[chan_id_col] = pd.to_numeric(ch[chan_id_col], errors="coerce").astype("Int64")
    ch[subs_col] = pd.to_numeric(ch[subs_col], errors="coerce").fillna(0).astype(float)
    id2subs = dict(zip(ch[chan_id_col], ch[subs_col]))

    out = df.copy()
    out[news_id_col] = pd.to_numeric(out[news_id_col], errors="coerce").astype("Int64")

    subs = out[news_id_col].map(id2subs).fillna(0.0).to_numpy(dtype=np.float32)
    log_subs = np.log1p(subs)

    mn, mx = float(log_subs.min()), float(log_subs.max())
    if mx > mn:
        w = (log_subs - mn) / (mx - mn)
    else:
        w = np.zeros_like(log_subs, dtype=np.float32)

    out[weight_col] = w.astype(np.float32)
    return out


In [26]:
ch_map = (channels[["id", "name"]]
          .dropna()
          .assign(id=lambda x: pd.to_numeric(x["id"], errors="coerce"))
          .dropna(subset=["id"])
          .assign(id=lambda x: x["id"].astype(int))
          .set_index("id")["name"]
          .to_dict())

df = df.copy()
df["id_channel"] = pd.to_numeric(df["id_channel"], errors="coerce")
df["channel_name"] = df["id_channel"].map(ch_map).fillna(df["id_channel"].astype("Int64").astype(str))
df = attach_channel_w(df, channels)

In [27]:
df.shape

(18618, 8)

In [28]:
import re
import numpy as np
import pandas as pd

def clean_news_text(s: str) -> str:
    s = s or ""
    s = re.sub(r"\s+", " ", s).strip()
    return s

def ensure_datetime(df: pd.DataFrame, col: str = "date") -> pd.DataFrame:
    d = df.copy()
    d[col] = pd.to_datetime(d[col], utc=True, errors="coerce")
    d = d.dropna(subset=[col])
    d["date_day"] = d[col].dt.floor("D")
    return d

df = ensure_datetime(df, "date")
df["message_id"] = df["message_id"].astype(str)
df["message"] = df["message"].fillna("").astype(str).map(clean_news_text)

In [29]:
from rank_bm25 import BM25Okapi

def tokenize_ru(text: str):
    text = text.lower()
    text = re.sub(r"[^0-9a-zа-яё\s]+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text.split()

corpus_tok = [tokenize_ru(t) for t in df["message"].tolist()]
bm25 = BM25Okapi(corpus_tok)

In [30]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "intfloat/multilingual-e5-large" 
encoder = SentenceTransformer(MODEL_NAME, device="cuda")  
texts = df["message"].tolist()
doc_inputs = ["passage: " + t for t in texts]

In [31]:
E_docs = encoder.encode(
    doc_inputs,
    batch_size=64,
    show_progress_bar=True,
    normalize_embeddings=True,
).astype(np.float32)

Batches:   0%|          | 0/291 [00:00<?, ?it/s]

In [32]:
import faiss

dim = E_docs.shape[1]
index = faiss.IndexFlatIP(dim)      
index.add(E_docs)

In [33]:
rowmap = df[["message_id","date","date_day","id_channel","channel_name"]].copy()

In [34]:
assert len(rowmap) == E_docs.shape[0]
assert len(rowmap) == index.ntotal

In [35]:
import re
import hashlib
import numpy as np
import pandas as pd

try:
    import faiss
except Exception:
    faiss = None

_URL_RE = re.compile(r"https?://\S+|www\.\S+")
_HANDLE_RE = re.compile(r"@\w+")
_WS_RE = re.compile(r"\s+")

def _normalize_for_dedup(text: str, mask_numbers: bool = True) -> str:
    if not isinstance(text, str):
        return ""
    t = text.lower()
    t = _URL_RE.sub(" ", t)
    t = _HANDLE_RE.sub(" ", t)
    t = re.sub(r"[^\w\s%.,\-]+", " ", t, flags=re.UNICODE)
    if mask_numbers:
        t = re.sub(r"\d+(?:[.,]\d+)?", "<num>", t)
    t = _WS_RE.sub(" ", t).strip()
    return t

def _stable_hash(s: str) -> str:
    return hashlib.md5(s.encode("utf-8", errors="ignore")).hexdigest()

def _union_find(n: int):
    parent = np.arange(n, dtype=np.int32)
    rank = np.zeros(n, dtype=np.int8)
    def find(x: int) -> int:
        while parent[x] != x:
            parent[x] = parent[parent[x]]
            x = parent[x]
        return int(x)
    def union(a: int, b: int) -> None:
        ra, rb = find(a), find(b)
        if ra == rb:
            return
        if rank[ra] < rank[rb]:
            parent[ra] = rb
        elif rank[ra] > rank[rb]:
            parent[rb] = ra
        else:
            parent[rb] = ra
            rank[ra] += 1
    return find, union

def dedup_cluster_candidates_time(
    cand: pd.DataFrame,
    encoder,
    text_col: str = "message",
    date_col: str = "date_day",
    channel_col: str = "channel_name",
    score_col: str = "score_rrf",
    sim_threshold: float = 0.95,
    knn: int = 20,
    keep_per_cluster: int = 1,
    mask_numbers: bool = True,
    max_day_diff: int = 1,
    overwrite_channel: bool = True,
    channel_join: str = "; ",
) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:

    if cand is None or len(cand) == 0:
        return cand, pd.DataFrame(), pd.DataFrame()

    cand = cand.copy().reset_index(drop=True)

    dts = pd.to_datetime(cand[date_col], errors="coerce", utc=True).dt.normalize()
    cand["_dt"] = dts
    cand["_dt_str"] = cand["_dt"].dt.strftime("%Y-%m-%d").fillna("")

    norm = cand[text_col].fillna("").map(lambda s: _normalize_for_dedup(s, mask_numbers=mask_numbers))
    cand["_h"] = norm.map(_stable_hash)

    cand["_hk"] = cand["_h"].astype(str) + "|" + cand["_dt_str"].astype(str)

    if score_col in cand.columns:
        rep_idx = (
            cand.sort_values(score_col, ascending=False)
                .groupby("_hk", as_index=False)
                .head(1)
                .index.to_numpy()
        )
    else:
        rep_idx = cand.groupby("_hk", as_index=False).head(1).index.to_numpy()

    rep = cand.loc[rep_idx].copy().reset_index(drop=True)

    texts = rep[text_col].fillna("").tolist()
    X = encoder.encode(texts, normalize_embeddings=True, show_progress_bar=False).astype(np.float32)

    rep_dt = pd.to_datetime(rep["_dt"], errors="coerce", utc=True)
    rep_dt = rep_dt.dt.tz_convert(None).dt.normalize().to_numpy(dtype="datetime64[D]")

    m = len(rep)
    find, union = _union_find(m)

    if m > 1:
        if faiss is None:
            S = X @ X.T
            for i in range(m):
                js = np.where(S[i, i+1:] >= sim_threshold)[0] + (i + 1)
                for j in js:
                    if np.isnat(rep_dt[i]) or np.isnat(rep_dt[j]):
                        continue
                    day_diff = abs(int((rep_dt[i] - rep_dt[j]).astype("timedelta64[D]").astype(int)))
                    if day_diff <= max_day_diff:
                        union(i, int(j))
        else:
            idx = faiss.IndexFlatIP(X.shape[1])
            idx.add(X)
            D, I = idx.search(X, min(knn, m))
            for i in range(m):
                for score, j in zip(D[i], I[i]):
                    if j < 0 or j == i:
                        continue
                    if float(score) < sim_threshold:
                        continue
                    if np.isnat(rep_dt[i]) or np.isnat(rep_dt[j]):
                        continue
                    day_diff = abs(int((rep_dt[i] - rep_dt[j]).astype("timedelta64[D]").astype(int)))
                    if day_diff <= max_day_diff:
                        union(i, int(j))

    rep_cluster = np.array([find(i) for i in range(m)], dtype=np.int32)
    _, rep_cluster = np.unique(rep_cluster, return_inverse=True)
    rep["_rep_cluster"] = rep_cluster

    hk_to_cluster = dict(zip(rep["_hk"].tolist(), rep["_rep_cluster"].tolist()))
    cand["_cluster_id"] = cand["_hk"].map(hk_to_cluster)

    cand["_cluster_id"] = cand["_cluster_id"].fillna(-1).astype(np.int32)

    cluster_sizes = cand.groupby("_cluster_id").size()

    if channel_col in cand.columns:
        ch_joined = (
            cand.groupby("_cluster_id")[channel_col]
                .apply(lambda s: channel_join.join(sorted({str(x) for x in s.dropna().tolist()})))
        )
    else:
        ch_joined = pd.Series(dtype=str)

    if score_col in cand.columns:
        cand_dedup = (
            cand.sort_values(score_col, ascending=False)
                .groupby("_cluster_id", group_keys=False)
                .head(keep_per_cluster)
                .reset_index(drop=True)
        )
    else:
        cand_dedup = (
            cand.groupby("_cluster_id", group_keys=False)
                .head(keep_per_cluster)
                .reset_index(drop=True)
        )

    cand_dedup["cluster_size"] = cand_dedup["_cluster_id"].map(cluster_sizes).astype(int)

    if channel_col in cand.columns:
        cand_dedup["channel_all"] = cand_dedup["_cluster_id"].map(ch_joined).fillna("")
        cand_dedup["channel_primary"] = cand_dedup[channel_col].astype(str)
        if overwrite_channel and channel_col in cand_dedup.columns:
            cand_dedup[channel_col] = cand_dedup["channel_all"]

    cand_dedup = cand_dedup.drop(columns=["_h", "_hk", "_dt", "_dt_str"], errors="ignore")

    clusters = cand[["_cluster_id"]].copy()
    clusters["cluster_size"] = clusters["_cluster_id"].map(cluster_sizes).astype(int)

    members = cand[["_cluster_id"]].copy()
    for c in ["date_day", "date", "channel_name", "channel", "message_id", "score_rrf"]:
        if c in cand.columns:
            members[c] = cand[c]
    members["text_snip"] = cand[text_col].fillna("").map(lambda s: s[:250])

    return cand_dedup, clusters, members

In [36]:
import numpy as np
import pandas as pd

def snippet(t: str, n: int = 1000) -> str:
    return t[:n]

def _topk_indices_from_scores(scores: np.ndarray, k: int) -> np.ndarray:
    k = min(k, len(scores))
    if k <= 0:
        return np.array([], dtype=int)
    if k == len(scores):
        idx = np.argsort(-scores)
    else:
        idx = np.argpartition(-scores, k - 1)[:k]
        idx = idx[np.argsort(-scores[idx])]
    return idx.astype(int)

def dense_candidates_faiss(index, encoder, query: str, topN: int = 500):
    qv = encoder.encode(
        ["query: " + query],
        normalize_embeddings=True,
        show_progress_bar=False
    ).astype(np.float32)
    scores, idx = index.search(qv, topN)
    return idx[0].astype(int), scores[0].astype(np.float32)

def _compute_time_arrays(df: pd.DataFrame, rowpos: np.ndarray, anchor_date, date_col: str):
    ad = pd.to_datetime(anchor_date, utc=True).normalize()
    dts = pd.to_datetime(df.loc[rowpos, date_col], errors="coerce", utc=True).dt.normalize()
    age = (ad - dts).dt.days.to_numpy(dtype=np.float32)
    age = np.where(np.isfinite(age), age, 1e9).astype(np.float32)
    age = np.where(age < 0, 1e9, age).astype(np.float32)
    return dts, age

def _time_rank_from_age(age_days: np.ndarray) -> np.ndarray:
    order = np.argsort(age_days, kind="stable")
    rank = np.empty_like(order, dtype=np.int32)
    rank[order] = np.arange(1, len(order) + 1, dtype=np.int32)
    return rank

def hybrid_retrieve_rrf(
    df: pd.DataFrame,
    index,
    encoder,
    bm25,
    tokenize_fn,
    query: str,
    k: int = 50,
    topN_each: int = 500,
    k_rrf: int = 60,
    w_dense: float = 1.0,
    w_bm25: float = 1.0,
    anchor_date: str | pd.Timestamp | None = None,
    date_col: str = "date_day",
    max_window_days: int | None = 365,
    w_time: float = 0.5,
    w_channel: float | None = None,
    channel_w_col: str = "channel_w",
) -> pd.DataFrame:
    if anchor_date is not None:
        ad = pd.to_datetime(anchor_date, utc=True).normalize()
        if date_col not in df.columns:
            raise KeyError(f"date_col='{date_col}' not found in df.columns")
        dts_all = pd.to_datetime(df[date_col], errors="coerce", utc=True).dt.normalize()
        allowed = (dts_all <= ad)
        if max_window_days is not None:
            age_all = (ad - dts_all).dt.days
            allowed &= (age_all >= 0) & (age_all <= int(max_window_days))
        allowed_np = allowed.to_numpy(dtype=bool)
    else:
        allowed_np = None

    d_idx, _ = dense_candidates_faiss(index, encoder, query, topN=topN_each)
    if allowed_np is not None and len(d_idx) > 0:
        d_idx = d_idx[allowed_np[d_idx]]
    dense_rank = {int(rowpos): r for r, rowpos in enumerate(d_idx, start=1)}

    if bm25 is None:
        union = d_idx.astype(int)
        if len(union) == 0:
            return df.iloc[[]].copy().reset_index(drop=True)

        rrf = w_dense / (k_rrf + np.arange(1, len(union) + 1, dtype=np.float32))

        rank_time = None
        if anchor_date is not None and w_time and len(union) > 0:
            _, age = _compute_time_arrays(df, union, anchor_date, date_col)
            rank_time = _time_rank_from_age(age)
            rrf = rrf + (w_time / (k_rrf + rank_time.astype(np.float32)))

        order = np.argsort(-rrf)
        union = union[order]
        rrf = rrf[order]
        if rank_time is not None:
            rank_time = rank_time[order]

        out = df.iloc[union].copy()
        out["_rowpos"] = union
        out["score_rrf"] = rrf
        out["rank_dense"] = out["_rowpos"].map(lambda rp: dense_rank.get(int(rp), np.nan))
        out["rank_bm25"] = np.nan

        if anchor_date is not None:
            doc_day, age = _compute_time_arrays(df, union, anchor_date, date_col)
            out["doc_day"] = doc_day.dt.tz_localize(None)
            out["age_days"] = age
            if rank_time is not None:
                out["rank_time"] = rank_time

        if channel_w_col in out.columns:
            if w_channel is None:
                w_channel = 0.10 * float(np.std(out["score_rrf"].to_numpy(dtype=np.float32)) or 1.0)
            out["score_rrf"] = out["score_rrf"] + float(w_channel) * out[channel_w_col].astype(np.float32)
            out = out.sort_values("score_rrf", ascending=False)

        return out.head(k).reset_index(drop=True)

    bm_scores = bm25.get_scores(tokenize_fn(query)).astype(np.float32)
    if allowed_np is not None:
        bm_scores[~allowed_np] = -np.inf
    b_idx = _topk_indices_from_scores(bm_scores, topN_each)
    bm_rank = {int(rowpos): r for r, rowpos in enumerate(b_idx, start=1)}

    union = np.array(sorted(set(dense_rank) | set(bm_rank)), dtype=int)
    if len(union) == 0:
        return df.iloc[[]].copy().reset_index(drop=True)

    rrf = np.zeros(len(union), dtype=np.float32)
    for j, rowpos in enumerate(union):
        if rowpos in dense_rank:
            rrf[j] += w_dense / (k_rrf + dense_rank[rowpos])
        if rowpos in bm_rank:
            rrf[j] += w_bm25 / (k_rrf + bm_rank[rowpos])

    rank_time = None
    if anchor_date is not None and w_time and len(union) > 0:
        _, age = _compute_time_arrays(df, union, anchor_date, date_col)
        rank_time = _time_rank_from_age(age)
        rrf = rrf + (w_time / (k_rrf + rank_time.astype(np.float32)))

    order = np.argsort(-rrf)
    union = union[order]
    rrf = rrf[order]
    if rank_time is not None:
        rank_time = rank_time[order]

    out = df.iloc[union].copy()
    out["_rowpos"] = union
    out["score_rrf"] = rrf
    out["rank_dense"] = out["_rowpos"].map(lambda rp: dense_rank.get(int(rp), np.nan))
    out["rank_bm25"] = out["_rowpos"].map(lambda rp: bm_rank.get(int(rp), np.nan))

    if anchor_date is not None:
        doc_day, age = _compute_time_arrays(df, union, anchor_date, date_col)
        out["doc_day"] = doc_day.dt.tz_localize(None)

        out["age_days"] = age
        if rank_time is not None:
            out["rank_time"] = rank_time

    if channel_w_col in out.columns:
        if w_channel is None:
            w_channel = 0.10 * float(np.std(out["score_rrf"].to_numpy(dtype=np.float32)) or 1.0)
        out["score_rrf"] = out["score_rrf"] + float(w_channel) * out[channel_w_col].astype(np.float32)
        out = out.sort_values("score_rrf", ascending=False)

    return out.head(k).reset_index(drop=True)

In [37]:
cand = hybrid_retrieve_rrf(
        df=df,
        index=index,
        encoder=encoder,
        bm25=bm25,
        tokenize_fn=tokenize_ru,
        query="Кофе в России может подорожать на 50% уже в этом году, предупреждают аналитики Last Wish. Причины — засуха в Бразилии, рост логистических расходов, дефицит запасов и разбалансировка мирового спроса. Больше всего прибавит в цене арабика.\n\n🔵 Bloomberg",
        k=50,
        topN_each=2000,
        k_rrf=60,
        w_dense=1.0,
        w_bm25=1.0,
        anchor_date="2025-09-04",
        max_window_days=365,
        w_time=0.6,
        w_channel=None)

In [38]:
cand.head()

,message_id,id_channel,message,date,topic,viral_final,channel_name,channel_w,date_day,_rowpos,score_rrf,rank_dense,rank_bm25,doc_day,age_days,rank_time
0,588a6960-aa5b-406b-8417-fdad3ccde1e9,5,Кофе в России может подорожать на 50% уже в эт...,2025-04-07 13:32:23+00:00,Сырьевые рынки,0.716038,Блумберг,0.063124,2025-04-07 00:00:00+00:00,6460,0.033057,1.0,1.0,2025-04-07,150.0,2326
1,4f13a7ff-e768-43ef-a71b-e429f3dada1b,4,Кофе может резко подорожать до 40% уже в этом ...,2025-03-24 06:48:20+00:00,Сырьевые рынки,0.588153,"Банки, деньги, два офшора",0.300566,2025-03-24 00:00:00+00:00,5729,0.032575,2.0,2.0,2025-03-24,164.0,2577
2,6ba48c2b-b64b-4326-aa39-05ebd207ca59,5,Россиян предупредили о резком скачке цен на ко...,2025-03-24 06:06:46+00:00,Сырьевые рынки,0.661343,Блумберг,0.063124,2025-03-24 00:00:00+00:00,7863,0.031504,5.0,3.0,2025-03-24,164.0,2579
3,8fbc1ae5-130c-4773-ba34-5e004af5861e,5,В России ожидается новое подорожание кофе — на...,2025-08-01 07:02:48+00:00,Сырьевые рынки,0.688021,Блумберг,0.063124,2025-08-01 00:00:00+00:00,10473,0.030596,3.0,14.0,2025-08-01,34.0,444
4,f7212d03-4fc9-4c70-b58c-bfb3336ff4ca,18,Цены на кофе и шоколад в этом году взлетят на ...,2025-01-09 16:17:44+00:00,Сырьевые рынки,0.771930,Экономика,0.009202,2025-01-09 00:00:00+00:00,17967,0.029833,10.0,5.0,2025-01-09,238.0,3696


In [39]:
def eval_one_retriever_temporal_rrf(
    df: pd.DataFrame,
    index,
    encoder,
    bm25,
    tokenize_fn,
    queries,
    k: int = 50,
    topN_each: int = 1000,
    k_rrf: int = 60,
    w_dense: float = 1.0,
    w_bm25: float = 1.0,
    batch_size: int = 32,

    # temporal params
    default_anchor_date: str | None = None,   # если anchor_date не лежит в qobj
    date_col: str = "date_day",
    max_window_days: int | None = 365,
    w_time: float = 0.6,
    w_channel: float | None = None,
    channel_w_col: str = "channel_w",
    method_name: str = "temporal_rrf",
):
    metric_rows = []
    pair_rows = []

    for qobj in tqdm(queries, desc=f"Queries ({method_name})"):
        q = qobj["text"]
        q_type = qobj.get("type", "unknown")
        exclude_mid = qobj.get("exclude_message_id", None)

        # поддержка per-query anchor_date: qobj["anchor_date"], иначе default
        anchor_date = qobj.get("anchor_date", default_anchor_date)

        cand = hybrid_retrieve_rrf(
            df=df,
            index=index,
            encoder=encoder,
            bm25=bm25,
            tokenize_fn=tokenize_fn,
            query=q,
            k=k,
            topN_each=topN_each,
            k_rrf=k_rrf,
            w_dense=w_dense,
            w_bm25=w_bm25,
            anchor_date=anchor_date,
            date_col=date_col,
            max_window_days=max_window_days,
            w_time=w_time,
            w_channel=w_channel,
            channel_w_col=channel_w_col,
        )

        if exclude_mid is not None and "message_id" in cand.columns:
            cand = cand[cand["message_id"].astype(str) != str(exclude_mid)]

        ids, docs = _take_top_docs(cand, k)
        rels = judge_pairs_batched([(q, d) for d in docs], batch_size=batch_size)

        metric_rows.append({
            "q_type": q_type,
            "query": q,
            "method": method_name,
            "ndcg@50": ndcg_at_k(rels, k),
            "mrr@50(rel=2)": mrr_at_k(rels, k, thr=2),
            "p@50(rel=2)": precision_at_k(rels, k, thr=2),
            "p@50(rel>=1)": precision_at_k(rels, k, thr=1),
            "n_docs": len(rels),
        })

        for r, (mid, rel) in enumerate(zip(ids, rels), start=1):
            pair_rows.append({
                "q_type": q_type,
                "query": q,
                "method": method_name,
                "rank": r,
                "message_id": mid,
                "relevance": int(rel),
            })

    return pd.DataFrame(metric_rows), pd.DataFrame(pair_rows)


In [40]:
new_metrics_df, new_pairs_df = eval_one_retriever_temporal_rrf(
    df=df,
    index=index,
    encoder=encoder,
    bm25=bm25,
    tokenize_fn=tokenize_ru,
    queries=queries,
    k=50,
    topN_each=2000,
    k_rrf=60,
    w_dense=1.0,
    w_bm25=1.0,
    batch_size=32,

    default_anchor_date="2025-09-04",
    max_window_days=365,
    w_time=0.6,
    w_channel=None,
)

new_summary = (new_metrics_df
               .groupby("method", as_index=False)[["ndcg@50","mrr@50(rel=2)","p@50(rel=2)","p@50(rel>=1)"]]
               .mean())
new_summary


Queries (temporal_rrf): 100%|██████████| 40/40 [02:53<00:00,  4.33s/it]


,method,ndcg@50,mrr@50(rel=2),p@50(rel=2),p@50(rel>=1)
0,temporal_rrf,0.773529,0.570238,0.265888,0.465388


In [41]:
metrics_all = pd.concat([metrics_df, new_metrics_df], ignore_index=True)
pairs_all   = pd.concat([pairs_df,   new_pairs_df],   ignore_index=True)

summary_all = (metrics_all
               .groupby("method", as_index=False)[["ndcg@50","mrr@50(rel=2)","p@50(rel=2)","p@50(rel>=1)"]]
               .mean())
summary_all


,method,ndcg@50,mrr@50(rel=2),p@50(rel=2),p@50(rel>=1)
0,bm25,0.739282,0.521766,0.195867,0.385082
1,hybrid,0.783114,0.593155,0.267857,0.469888
2,semantic,0.763042,0.572750,0.280347,0.484347
3,temporal_rrf,0.773529,0.570238,0.265888,0.465388


In [43]:
by_type = (metrics_all
           .groupby(["q_type", "method"])[["ndcg@50","mrr@50(rel=2)","p@50(rel=2)","p@50(rel>=1)"]]
           .mean()
           .sort_index())
display(by_type)

ndcg@50  mrr@50(rel=2)  p@50(rel=2)  p@50(rel>=1)
q_type method                                                          
news   bm25          0.650798       0.329167     0.036735      0.158163
       hybrid        0.660490       0.287500     0.035714      0.188776
       semantic      0.612732       0.239251     0.034694      0.184694
       temporal_rrf  0.654856       0.304167     0.038776      0.188776
topic  bm25          0.827766       0.714365     0.355000      0.612000
       hybrid        0.905737       0.898810     0.500000      0.751000
       semantic      0.913351       0.906250     0.526000      0.784000
       temporal_rrf  0.892201       0.836310     0.493000      0.742000